In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
# Deafen the particular pandas warnings
pd.options.mode.chained_assignment = None

In [2]:
# Import the datasets
features_0 = pd.read_csv("Features.csv")
features_1 = pd.read_csv("Features2022.csv")
features_2 = pd.read_csv("Features2022_1.csv")
energy_1 = pd.read_csv("Energy2022.csv")
energy_2 = pd.read_csv("Energy2022_1.csv")
energy_0 = pd.read_excel("Energy.xlsx", header = 1)
humidity = pd.read_csv("Humidity.csv")
irradiance = pd.read_csv("Irradiance.csv")
energy_2.rename(columns={'Time': 'Aika',
                        }, inplace=True)
features = pd.concat([features_0, features_1, features_2])
energy = pd.concat([energy_1, energy_2])

In [3]:
features

,Time,_S_talo_S_LV02_TE03,_S_talo_S_LV01_TE03,_S_talo_S_LIV01_TE03,_S_talo_U_TE90,_S_talo_S_KLV01_TE01,_S_talo_S_LKV01_TV01,_S_talo_S_LKV01_TE03,_S_talo_S111_TE,_S_talo_S112_TE,...,_S_talo_S214b_QE,_S_talo_S215_QE,_S_talo_S301_QE,_S_talo_S307_QE,_S_talo_S310_QE,_S_talo_S311_QE,_S_talo_S311b_QE,_S_talo_S312_QE,_S_talo_S313_QE,_S_talo_S316_QE
0,2020-01-09 13:00:00,28.2,38.1,40.5,1.3000,82.3,29.4,57.5,21.5,20.8,...,0.6330,435.0,442.0,536.0,516.0,552.0,521.0,632.0,441.0,567.0
1,2020-01-09 14:00:00,28.2,38.3,38.1,0.9440,82.4,30.0,58.0,21.5,20.8,...,0.6860,428.0,444.0,578.0,479.0,486.0,505.0,475.0,434.0,586.0
2,2020-01-09 15:00:00,28.6,39.5,39.3,0.6350,82.5,30.0,58.0,21.5,20.8,...,0.7220,420.0,423.0,498.0,436.0,445.0,427.0,429.0,424.0,483.0
3,2020-01-09 16:00:00,28.0,38.6,38.7,0.3680,79.9,26.0,58.1,21.5,20.8,...,0.6770,416.0,420.0,443.0,437.0,443.0,420.0,423.0,420.0,418.0
4,2020-01-09 17:00:00,28.6,39.5,39.0,0.0586,77.2,28.4,58.0,21.4,20.8,...,0.7070,416.0,419.0,426.0,421.0,426.0,418.0,421.0,420.0,416.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,2022-02-14 02:00:00,28.0,36.4,37.2,2.1400,79.4,29.6,58.0,21.6,21.0,...,0.2130,414.0,418.0,416.0,415.0,416.0,417.0,417.0,420.0,415.0
110,2022-02-14 03:00:00,27.5,36.7,37.5,2.0900,79.1,29.9,58.0,21.6,21.0,...,0.0838,414.0,419.0,416.0,414.0,416.0,417.0,418.0,420.0,414.0
111,2022-02-14 04:00:00,28.0,37.1,37.5,2.1900,79.2,31.0,58.0,21.6,20.9,...,0.0958,418.0,420.0,420.0,417.0,419.0,421.0,420.0,422.0,420.0
112,2022-02-14 05:00:00,27.7,36.4,37.2,2.1400,79.5,29.1,57.9,21.6,20.9,...,0.2830,418.0,423.0,422.0,419.0,421.0,425.0,422.0,423.0,419.0


In [4]:
energy_0["Aika"] = energy_0["Aika"].str.replace("\+3", '', regex = True)
energy_0["Aika"] = energy_0["Aika"].str[3:-1]
energy_0["Aika"] = pd.to_datetime(energy_0.Aika, format = '%d.%m.%Y %H.%M.%S')
energy_0["Time"] = energy_0["Aika"].dt.tz_localize('Europe/Helsinki', ambiguous='infer').dt.tz_convert('UTC')

energy["Aika"] = pd.to_datetime(energy.Aika, format = '%Y-%m-%d %H:%M:%S')
energy["Time"] = energy["Aika"].dt.tz_localize('Europe/Helsinki', ambiguous='infer').dt.tz_convert('UTC')

energy = pd.concat([energy, energy_0]).drop_duplicates().sort_values("Time").reset_index(drop=True)

In [5]:
energy

,Aika,kWh,Time
0,2018-01-01 00:00:00,73,2017-12-31 22:00:00+00:00
1,2018-01-01 01:00:00,73,2017-12-31 23:00:00+00:00
2,2018-01-01 02:00:00,58,2018-01-01 00:00:00+00:00
3,2018-01-01 03:00:00,57,2018-01-01 01:00:00+00:00
4,2018-01-01 04:00:00,59,2018-01-01 02:00:00+00:00
...,...,...,...
37353,2022-02-14 02:00:00,61,2022-02-14 00:00:00+00:00
37354,2022-02-14 03:00:00,57,2022-02-14 01:00:00+00:00
37355,2022-02-14 04:00:00,61,2022-02-14 02:00:00+00:00
37356,2022-02-14 05:00:00,59,2022-02-14 03:00:00+00:00


In [6]:
# From the features, select the observations with temperatures and CO2 concentration
temperatures = features[['_S_talo_S111_TE',
       '_S_talo_S112_TE', '_S_talo_S114_TE', '_S_talo_S115_TE',
       '_S_talo_S116_TE', '_S_talo_S117_TE', '_S_talo_S119_TE',
       '_S_talo_S125_TE', '_S_talo_S201_TE', '_S_talo_S202_TE',
       '_S_talo_S210_TE', '_S_talo_S213_TE', '_S_talo_S213b_TE',
       '_S_talo_S214_TE', '_S_talo_S214b_TE', '_S_talo_S215_TE',
       '_S_talo_S301_TE', '_S_talo_S307_TE', '_S_talo_S310_TE',
       '_S_talo_S311_TE', '_S_talo_S311b_TE', '_S_talo_S312_TE',
       '_S_talo_S313_TE', '_S_talo_S316_TE']]

qualities = features[['_S_talo_S111_QE', '_S_talo_S112_QE', '_S_talo_S114_QE',
       '_S_talo_S115_QE', '_S_talo_S116_QE', '_S_talo_S117_QE',
       '_S_talo_S119_QE', '_S_talo_S125_QE', '_S_talo_S201_QE',
       '_S_talo_S202_QE', '_S_talo_S210_QE', '_S_talo_S213_QE',
       '_S_talo_S213b_QE', '_S_talo_S214_QE', '_S_talo_S214b_QE',
       '_S_talo_S215_QE', '_S_talo_S301_QE', '_S_talo_S307_QE',
       '_S_talo_S310_QE', '_S_talo_S311_QE', '_S_talo_S311b_QE',
       '_S_talo_S312_QE', '_S_talo_S313_QE', '_S_talo_S316_QE']]

# Multiply some room observations of CO2 concentration to normalize to them scale
for i in ["_S_talo_S111_QE", "_S_talo_S114_QE", "_S_talo_S115_QE",
          "_S_talo_S116_QE", "_S_talo_S117_QE", "_S_talo_S119_QE", 
          "_S_talo_S213b_QE", "_S_talo_S214b_QE"]:
    qualities.loc[:, i] = qualities.loc[:, i] * 1000
    
# Calculate the mean of the room observations  
temperatures.loc[:, "Inside_temperature"] = temperatures.mean(1)   
qualities.loc[:, "CO2_concentration"] = qualities.mean(1)

# Select and rename the remaining sensors
features = features[['Time', '_S_talo_S_LV02_TE03', '_S_talo_S_LV01_TE03',
       '_S_talo_S_LIV01_TE03', '_S_talo_U_TE90', '_S_talo_S_KLV01_TE01',
       '_S_talo_S_LKV01_TV01', '_S_talo_S_LKV01_TE03']]

features.rename(columns={'_S_talo_S_LV02_TE03': 'Floor_network_2_temperature', 
                         '_S_talo_S_LV01_TE03': 'Radiator_network_1_temperature',
                         "_S_talo_S_LIV01_TE03": "Ventilation_network_1_temperature",
                         "_S_talo_U_TE90": "Outside_temperature_1",
                         "_S_talo_S_KLV01_TE01": "District_heat_temperature",
                         "_S_talo_S_LKV01_TV01": "Domestic_water_network_1_primary_valve",
                         "_S_talo_S_LKV01_TE03": "Domestic_water_network_1_temperature"
                        }, inplace=True)

# Merge the temperature, CO2 concentration and other features into the same dataset
features = pd.concat([features, temperatures["Inside_temperature"], qualities["CO2_concentration"]], axis = 1)

# Set the correct timezone (GMT) for the feature data
features["Time"] = pd.to_datetime(features.Time, format = '%Y-%m-%d %H:%M:%S')
features["Time"] = features["Time"].dt.tz_localize('UTC', ambiguous='infer')

# Set the correct timezone (Finnish) for the energy data
#energy["Aika"] = pd.to_datetime(energy.Aika, format = '%Y-%m-%d %H:%M:%S')
#energy["Time"] = energy["Aika"].dt.tz_localize('Europe/Helsinki', ambiguous='infer').dt.tz_convert('UTC')

# Merge the features and the energy into single dataset
result = pd.merge(features, energy[["kWh", "Time"]], how = "left", on = "Time")

# Rename the energy column
result.rename(columns = {"kWh" : "Energy_consumption"}, inplace = True)

# Merge the humidity and irradiance datasets with the general dataset
def data_merge(main_dataset, dataset, parameter):
    dataset["Aika"] = dataset["Year"].astype(str) + "-" + dataset["m"].astype(str) + "-" + dataset["d"].astype(str) + " " + dataset["Time"]
    dataset["Aika"] = pd.to_datetime(dataset.Aika, format = '%Y.%m.%d %H:%M')
    dataset["Time"] = dataset["Aika"].dt.tz_localize('UTC', ambiguous='infer')
    main_dataset = pd.merge(main_dataset, dataset[[parameter, "Time"]], how = "left", on = "Time")
    return main_dataset

result = data_merge(result, humidity, "Relative humidity (%)")
result = data_merge(result, irradiance, "Direct solar radiation (W/m2)")

# Rename the remaining columns
result.rename(columns = {"Direct solar radiation (W/m2)" : "Solar_irradiance", 
                           "Relative humidity (%)" : "Outside_humidity", 
                           "Outside_temperature_1" : "Outside_temperature_average"}, inplace = True)

# Detect and remove the missing window when all sensors show zero
datana = result[(result["Outside_temperature_average"] == 0) & (result["Floor_network_2_temperature"] == 0)]
result = result.drop(index = datana.index)
result.reset_index(inplace = True)

#Drop inplaced index
result = result.drop(columns = ["index"])

# Drop NAs
result = result.drop(index = result[result.isna().any(axis=1)].index)

# Remove observations when the network was under maintenance
result = result.drop(result[(result.Time >= "2022-02-11 12:00:00") & (result.Time < "2022-02-11 17:00:00")].index)

In [7]:
result = result.iloc[-2700:].reset_index()

In [8]:
result

,index,Time,Floor_network_2_temperature,Radiator_network_1_temperature,Ventilation_network_1_temperature,Outside_temperature_average,District_heat_temperature,Domestic_water_network_1_primary_valve,Domestic_water_network_1_temperature,Inside_temperature,CO2_concentration,Energy_consumption,Outside_humidity,Solar_irradiance
0,10809,2021-11-30 12:00:00+00:00,32.3,53.8,49.6,-10.00,91.6,29.9,57.8,21.883333,512.291667,109.0,83.0,3.5
1,10810,2021-11-30 13:00:00+00:00,31.7,52.9,49.1,-11.00,91.3,28.8,58.1,21.837500,486.125000,107.0,84.0,0.7
2,10811,2021-11-30 13:00:00+00:00,31.7,52.9,49.1,-11.00,91.3,28.8,58.1,21.837500,486.125000,108.0,84.0,0.7
3,10812,2021-11-30 14:00:00+00:00,32.2,53.8,49.7,-11.00,91.4,27.4,58.1,21.770833,460.000000,111.0,85.0,0.4
4,10813,2021-11-30 14:00:00+00:00,32.2,53.8,49.7,-11.00,91.4,27.4,58.1,21.770833,460.000000,114.0,85.0,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,13842,2022-02-14 00:00:00+00:00,27.9,37.0,37.3,1.84,79.5,27.4,58.0,21.262500,442.250000,61.0,94.0,1.3
2696,13843,2022-02-14 01:00:00+00:00,27.5,37.5,37.4,1.96,79.6,28.2,58.0,21.279167,447.083333,57.0,95.0,1.3
2697,13844,2022-02-14 02:00:00+00:00,28.0,36.4,37.2,2.14,79.4,29.6,58.0,21.262500,441.416667,61.0,96.0,0.9
2698,13845,2022-02-14 03:00:00+00:00,27.5,36.7,37.5,2.09,79.1,29.9,58.0,21.254167,440.075000,59.0,96.0,1.3


In [9]:
(result["Time"].iloc[8] - result["Time"].iloc[0]).components.hours

4

In [13]:
# Create a temperature delay column
def temp_delay(delay):
    result["Delayed_temperature"] = result["Inside_temperature"]
    for i in result.index.values[:-(delay)]:
        if (result["Time"].iloc[(i + delay)] - result["Time"].iloc[i]).components.hours == delay:
            result["Delayed_temperature"][i] = result["Inside_temperature"].iloc[i:i + delay + 1].shift(-delay)
    
temp_delay(4)

In [15]:
result

,index,Time,Floor_network_2_temperature,Radiator_network_1_temperature,Ventilation_network_1_temperature,Outside_temperature_average,District_heat_temperature,Domestic_water_network_1_primary_valve,Domestic_water_network_1_temperature,Inside_temperature,CO2_concentration,Energy_consumption,Outside_humidity,Solar_irradiance,Delayed_temperature
0,10809,2021-11-30 12:00:00+00:00,32.3,53.8,49.6,-10.00,91.6,29.9,57.8,21.883333,512.291667,109.0,83.0,3.5,21.883333
1,10810,2021-11-30 13:00:00+00:00,31.7,52.9,49.1,-11.00,91.3,28.8,58.1,21.837500,486.125000,107.0,84.0,0.7,21.837500
2,10811,2021-11-30 13:00:00+00:00,31.7,52.9,49.1,-11.00,91.3,28.8,58.1,21.837500,486.125000,108.0,84.0,0.7,21.837500
3,10812,2021-11-30 14:00:00+00:00,32.2,53.8,49.7,-11.00,91.4,27.4,58.1,21.770833,460.000000,111.0,85.0,0.4,21.770833
4,10813,2021-11-30 14:00:00+00:00,32.2,53.8,49.7,-11.00,91.4,27.4,58.1,21.770833,460.000000,114.0,85.0,0.4,21.770833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,13842,2022-02-14 00:00:00+00:00,27.9,37.0,37.3,1.84,79.5,27.4,58.0,21.262500,442.250000,61.0,94.0,1.3,21.233333
2696,13843,2022-02-14 01:00:00+00:00,27.5,37.5,37.4,1.96,79.6,28.2,58.0,21.279167,447.083333,57.0,95.0,1.3,21.279167
2697,13844,2022-02-14 02:00:00+00:00,28.0,36.4,37.2,2.14,79.4,29.6,58.0,21.262500,441.416667,61.0,96.0,0.9,21.262500
2698,13845,2022-02-14 03:00:00+00:00,27.5,36.7,37.5,2.09,79.1,29.9,58.0,21.254167,440.075000,59.0,96.0,1.3,21.254167


In [53]:
result["Inside_temperature"].iloc[0:5].shift(-4)[0]

21.770833333333332

In [54]:
result = result.dropna()

In [55]:
result.iloc[0]

index                                                         10809
Time                                      2021-11-30 12:00:00+00:00
Floor_network_2_temperature                                    32.3
Radiator_network_1_temperature                                 53.8
Ventilation_network_1_temperature                              49.6
Outside_temperature_average                                   -10.0
District_heat_temperature                                      91.6
Domestic_water_network_1_primary_valve                         29.9
Domestic_water_network_1_temperature                           57.8
Inside_temperature                                        21.883333
CO2_concentration                                        512.291667
Energy_consumption                                            109.0
Outside_humidity                                               83.0
Solar_irradiance                                                3.5
Delayed_temperature                             

In [56]:
# Save data to csv
result.to_csv("s-building_data.csv", index = False)